In [1]:
import pandas as pd
import numpy as np

# Text Methods

In [2]:
mystring = 'hello'
mystring.capitalize()

'Hello'

In [3]:
mystring.isdigit()

False

In [4]:
#help(str)

## Text Methods on Pandas String Column

Pandas can do a lot more than what we show here. Full online documentation on things like advanced string indexing and regular expressions with pandas can be found here: https://pandas.pydata.org/docs/user_guide/text.html

In [5]:
names = pd.Series(['andrew','bobo','claire','david','4'])
names

0    andrew
1      bobo
2    claire
3     david
4         4
dtype: object

In [6]:
names.str.capitalize()

0    Andrew
1      Bobo
2    Claire
3     David
4         4
dtype: object

In [7]:
names.str.isdigit()

0    False
1    False
2    False
3    False
4     True
dtype: bool

## Splitting , Grabbing, and Expanding

In [8]:
tech_finance = ['GOOG,APPL,AMZN','JPM,BAC,GS']
len(tech_finance)

2

In [9]:
tickers = pd.Series(tech_finance)
tickers

0    GOOG,APPL,AMZN
1        JPM,BAC,GS
dtype: object

In [10]:
tickers.str.split(',')

0    [GOOG, APPL, AMZN]
1        [JPM, BAC, GS]
dtype: object

In [11]:
tickers.str.split(',').str[0]

0    GOOG
1     JPM
dtype: object

In [12]:
tickers.str.split(',',expand=True)

,0,1,2
0,GOOG,APPL,AMZN
1,JPM,BAC,GS


## Cleaning or Editing Strings

In [13]:
messy_names = pd.Series(["andrew  ","bo;bo","  claire  "])
messy_names

0      andrew  
1         bo;bo
2      claire  
dtype: object

In [14]:
messy_names.str.replace(";","")

0      andrew  
1          bobo
2      claire  
dtype: object

In [15]:
messy_names.str.strip()

0    andrew
1     bo;bo
2    claire
dtype: object

In [16]:
messy_names.str.replace(";","").str.strip()

0    andrew
1      bobo
2    claire
dtype: object

In [17]:
messy_names.str.replace(";","").str.strip().str.capitalize()

0    Andrew
1      Bobo
2    Claire
dtype: object

## Alternative with Custom apply() call

In [18]:
def cleanup(name):
    name = name.replace(";","")
    name = name.strip()
    name = name.capitalize()
    return name

In [19]:
names = messy_names.apply(cleanup)
names

0    Andrew
1      Bobo
2    Claire
dtype: object